# Profitable App Profiles for the App Store and Google Play Markets

This article was written as a part of my [Data Science Challenge](https://note.com/naokiwifruit/n/n986686e6a101) and a project of ["Python for Data Science: Fundamentals"](https://www.dataquest.io/course/python-for-data-science-fundamentals/) at Dataquest.

We analyze the dataset for mobile apps in Google Play and App Store in order to find apps profile that is profitable.

## The Goal of This Project

The analysis is conducted on the assumption that you are working as a data scientist at a company that develops mobile apps.

The company only develops free apps, and the main source of revenue is in-app advertising. In other words, the more users who use your app, the more revenue you will earn.

The purpose of this project is to analyze app datasets and help our developers understand the types of apps that attract users in the market.
We are going to develop apps for both Android and iOS, so we will explore these two markets.

## App Datasets

* [A dataset](https://www.kaggle.com/lava18/google-play-store-apps) for 10,000 Android apps from the Google Play
* [A dataset](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) for 7,000 iOS apps from the App store

## Opening Data

In [1]:
from csv import reader

def open_dataset(file_path):
    """
    read a csv file and return data as a list
    
    Args:
        file_path (str) : the path to a csv file
    
    Returns:
        data (list) : data as a list of lists
    """
    
    opened_file = open(file_path, encoding='utf-8')
    read_file = reader(opened_file)
    data = list(read_file)
    
    return data

# The Google Play dataset
android = open_dataset('../data/googleplaystore.csv')
android_header = android[0]
android = android[1:]

# The App Store dataset
ios = open_dataset('../data/AppleStore.csv')
ios_header = ios[0]
ios = ios[1:]

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    """
    show information of rows and columns of the dataset
    
    Args:
        dataset (list) : data as a list of lists
        start (int) : starting indices of a slice from the data set
        end (int) : ending indices of a slice from the data set
        rows_and_columns (bool) : print the number of rows and columns if True
    """
    
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
        
    if rows_and_columns:
        print('Number of rows', len(dataset))
        print('Number of columns:', len(dataset[0]))
        
# print information of each dataset
print('====================')
print('Android')
print('====================\n')
print(android_header)
print('\n')
explore_data(android, 0, 1, True)

print('\n====================')
print('iOS')
print('====================\n')
print(ios_header)
print('\n')
explore_data(ios, 0, 1, True)

Android

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows 10841
Number of columns: 13

iOS

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


Number of rows 7197
Number of columns: 16


## Remove Incorrect Entry

In [3]:
# print the row to check whetherr is is indeed incorrect
print(android_header) # header
print('\n')
print(android[1]) # correct row
print('\n')
print(android[10472])# incorrect row

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


In [4]:
# remove the row
del android[10472]

## Remove Duplicate Entry

In [5]:
# print duplicate rows in the Google Play data set
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps;', duplicate_apps[:15])

Number of duplicate apps: 1181


Examples of duplicate apps; ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


In [6]:
# print all entries for Twitter
for app in android:
    name = app[0]
    if name == 'Twitter':
        print(app)

['Twitter', 'NEWS_AND_MAGAZINES', '4.3', '11667403', 'Varies with device', '500,000,000+', 'Free', '0', 'Mature 17+', 'News & Magazines', 'August 6, 2018', 'Varies with device', 'Varies with device']
['Twitter', 'NEWS_AND_MAGAZINES', '4.3', '11667403', 'Varies with device', '500,000,000+', 'Free', '0', 'Mature 17+', 'News & Magazines', 'August 6, 2018', 'Varies with device', 'Varies with device']
['Twitter', 'NEWS_AND_MAGAZINES', '4.3', '11657972', 'Varies with device', '500,000,000+', 'Free', '0', 'Mature 17+', 'News & Magazines', 'July 30, 2018', 'Varies with device', 'Varies with device']


In [7]:
# create a dictionary where each key is a unique app name and the corresponding value is the highest number of reviews of that app
reviews_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3]) # the 'Reviews' column
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
                
# use the dictionary and create a new data set which will have only one entry per app
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)

print('Expected length:', len(android) - len(duplicate_apps))
print('Actual length:', len(android_clean))

Expected length: 9659
Actual length: 9659


## Extract English App

In [8]:
def detect_english(name):
    """
    if app's name has more than three non-english characters, return False
    
    Args:
        name (str) : app's name
        
    Returns:
        bool : whether the app is for the non-English users
    """
    num_non_eng = 0
    for character in name:
        if ord(character) > 127: # according to ASCII, English text are all in the range 0 to 127
            num_non_eng += 1
            
    return False if num_non_eng > 3 else True

# make a list only contains English Android app
android_eng = []
for app in android_clean:
    name = app[0]
    if detect_english(name):
        android_eng.append(app)
        
# make a list only contains Enlgish iOS app
ios_eng = []
for app in ios:
    name = app[1]
    if detect_english(name):
        ios_eng.append(app)
        
print('The length of all Android apps:', len(android_clean))
print('The length of English Android apps:', len(android_eng))
print('\n')
print('The length of all iOS apps:', len(ios))
print('The length of English iOS apps:', len(ios_eng))

The length of all Android apps: 9659
The length of English Android apps: 9614


The length of all iOS apps: 7197
The length of English iOS apps: 6183


# Extract Free App

In [9]:
# isolate the free apps
android_free = []
for app in android_eng:
    if app[7] == '0':
        android_free.append(app)
        
ios_free = []
for app in ios_eng:
    if app[4] == '0.0':
        ios_free.append(app)
        
print('The length of English Android apps:', len(android_eng))
print('The length of free English Android apps:', len(android_free))
print('\n')
print('The length of English iOS apps:', len(ios_eng))
print('The length of free English iOS apps:', len(ios_free))

The length of English Android apps: 9614
The length of free English Android apps: 8864


The length of English iOS apps: 6183
The length of free English iOS apps: 3222


## Find Most Common Apps by Genre

In [10]:
def freq_table(dataset, index):
    """
    make a frequency table for a column you want as a dictionary
    
    Args:
        dataset (list) : a dataset as a list of lists 
        index (int) : a column you want to make the frequency table for 
        
    Returns:
        table (dic) : a frequency table for a column you want
    """
    
    table = {}
    total = 0
    #
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
            
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage
        
    return table_percentages
            
    
def display_table(dataset, index):
    """
    prints the entries of the frequency table in descending order
    
    Args:
        dataset (list) : a dataset as a list of lists 
        index (int) : a column you want to show the frequency table
    """
    
    table = freq_table(dataset, index)
    table_display = []
    # convert a dictionaly to tuple that can be sorted
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    # shot the table in descending order
    table_sorted = sorted(table_display, reverse=True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [11]:
# display the frequency table of 'Genres' column of the Google Play data set
display_table(android_free, -4)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

In [12]:
# display the frequency table of 'Category' column of the Google Play data set
display_table(android_free, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [13]:
# display the frequency table of 'prime_genre' column in the App Store data set
display_table(ios_free, 11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


## Most Popular Apps by Genre on the Google Play

In [14]:
# show the average number of install per app genre on the Google Play data set
for category in freq_table(android_free, 1):
    total = 0
    len_category = 0
    for app in android_free:
        category_app = app[1]
        if category_app == category:
            n_installs = app[5]
            n_installs = n_installs.replace('+', '')
            n_installs = n_installs.replace(',', '')
            n_installs = float(n_installs)
            total += n_installs
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

In [15]:
from statistics import median

# show the median of the number of installs per app genre on the Google Play data set
for category in freq_table(android_free, 1):
    n_installs_list = []
    for app in android_free:
        category_app = app[1]
        
        if category_app == category:
            n_installs = app[5]
            n_installs = n_installs.replace('+', '')
            n_installs = n_installs.replace(',', '')
            n_installs = float(n_installs)
            n_installs_list.append(n_installs)
    med_installs = median(n_installs_list)
    print(category, ':', med_installs)

ART_AND_DESIGN : 100000.0
AUTO_AND_VEHICLES : 100000.0
BEAUTY : 50000.0
BOOKS_AND_REFERENCE : 50000.0
BUSINESS : 1000.0
COMICS : 100000.0
COMMUNICATION : 500000.0
DATING : 10000.0
EDUCATION : 1000000.0
ENTERTAINMENT : 1000000.0
EVENTS : 1000.0
FINANCE : 10000.0
FOOD_AND_DRINK : 500000.0
HEALTH_AND_FITNESS : 500000.0
HOUSE_AND_HOME : 500000.0
LIBRARIES_AND_DEMO : 10000.0
LIFESTYLE : 10000.0
GAME : 1000000.0
FAMILY : 100000.0
MEDICAL : 1000.0
SOCIAL : 100000.0
SHOPPING : 1000000.0
PHOTOGRAPHY : 1000000.0
SPORTS : 100000.0
TRAVEL_AND_LOCAL : 100000.0
TOOLS : 100000.0
PERSONALIZATION : 100000.0
PRODUCTIVITY : 100000.0
PARENTING : 100000.0
WEATHER : 1000000.0
VIDEO_PLAYERS : 1000000.0
NEWS_AND_MAGAZINES : 50000.0
MAPS_AND_NAVIGATION : 100000.0


## Most Popular Apps by Genre on the App Store

In [16]:
# show the average number of user rating per app genre on the App Store
for genre in freq_table(ios_free, -5):
    total = 0
    len_genre = 0
    for app in ios_free:
        genre_app = app[11]
        if genre_app == genre:
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


In [17]:
# show the median of the number of ratings per app genre on the App Store data set
for genre in freq_table(ios_free, -5):
    n_ratings_list = []
    for app in ios_free:
        genre_app = app[-5]
        if genre_app == genre:
            n_ratings = float(app[5])
            n_ratings_list.append(n_ratings)
    med_ratings = median(n_ratings_list)
    print(genre, ':', med_ratings)

Social Networking : 4199.0
Photo & Video : 2206.0
Games : 901.5
Music : 3850.0
Reference : 6614.0
Health & Fitness : 2459.0
Weather : 289.0
Utilities : 1110.0
Travel : 798.5
Shopping : 5936.0
News : 373.0
Navigation : 8196.5
Lifestyle : 1111.0
Entertainment : 1197.5
Food & Drink : 1490.5
Sports : 1628.0
Book : 421.5
Finance : 1931.0
Education : 606.5
Productivity : 8737.5
Business : 1150.0
Catalogs : 1229.0
Medical : 566.5


In [18]:
# show the app names and the number of revies in the 'Navigation'
for app in ios_free:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


In [19]:
# show the app names and the number of revies in the 'Reference'
for app in ios_free:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


In [20]:
# show the app names and the number of revies in the 'Shopping'
for app in ios_free:
    if app[-5] == 'Shopping':
        print(app[1], ':', app[5])

Groupon - Deals, Coupons & Discount Shopping App : 417779
eBay: Best App to Buy, Sell, Save! Online Shopping : 262241
Wish - Shopping Made Fun : 141960
shopkick - Shopping Rewards & Discounts : 130823
Amazon App: shop, scan, compare, and read reviews : 126312
Target : 108131
Zappos: shop shoes & clothes, fast free shipping : 103655
Walgreens – Pharmacy, Photo, Coupons and Shopping : 88885
Best Buy : 80424
Walmart: Free 2-Day Shipping,* Easy Store Shopping : 70286
OfferUp - Buy. Sell. Simple. : 57348
Apple Store : 55171
Shop Savvy Barcode Scanner - Price Compare & Deals : 54630
Ibotta: Cash Back App, Grocery Coupons & Shopping : 44313
letgo: Buy & Sell Second Hand Stuff : 38424
CVS Pharmacy : 35981
Victoria’s Secret – The Sexiest Bras & Lingerie : 34507
Etsy: Shop Handmade, Vintage & Creative Goods : 30434
Gilt : 26353
Mercari: Shopping Marketplace to Buy & Sell Stuff : 24244
Shopular Coupons, Weekly Deals for Target, Walmart : 22729
RetailMeNot Shopping Deals, Coupons, Savings : 18544


## Conclusion

In this notebook, we analyzed the Android and iOS app data and investigated the most common or popular genres. While the Android market has a broad range of popular genres, but an entertainment-related app was popular in the iOS market. In terms of popularity, navigation apps, book apps, and shopping apps showed a strong presence. We concluded that developing a unique app for these three genres can be profitable for both the Android and iOS markets.

There are many points to be improved, such as scrutinizing the results of each genre and taking into account development costs, but we were here able to learn the Python basics and a data science workflow by analyzing the mobile app data.